In [1]:
import tensorflow as tf
import keras 
from tqdm import tqdm
import re
import pandas as pd 
import numpy as np
import random as rn
import os
print(keras.__version__)
print(tf.__version__)

# Utiliser un seed fixé pour le générateur de nombres aléatoires afin de résoudre le problème du caractère aléatoire et d'obtenir des résultats reproductibles avec Kéras. Les nombres ne font pas beaucoup de différence.
seed = 42
np.random.seed(seed)

2.3.1
1.15.0


Using TensorFlow backend.


## Prétraitement

In [2]:
from tqdm import tqdm


phrases=[]
mots = []
cibles=[] 
phr_position=0 

labels= {'Correct':0, 'ArtOrDet':1, 'Nn':2, 'Prep':3, 'SVA':4, 'Vform':5, 'Vt':6, 'Wform':7}

 
# Importer les phrases du fichier de données
f=open('/Users/highsierra/Tech/Labortory/ML-Fundamentals/release3.2/data/conll14st-preprocessed.m2')
for ligne in tqdm(f):
    parties = ligne.split()
    if(len(parties)>0):
        if ligne[0]=='S':
        # Traitement préliminaire des entrées   
            
            # Nettoyage
            ligne = ligne[2:].strip()
            
            # Liste de phrases
            phrases.append(ligne)

            # Liste de mots, pour en déduire la liste de vocabulaire
            mots = mots + ligne.split()

            
        # Traitement préliminaire des sorties   
            
            # Par défaut, considérer chaque mot comme non erroné, en créant un tableau avec l'étiquette "Correct" (i.e. sa valeur "0") pour chaque mot.
            etiquettes=np.zeros(shape=(len(parties)-1), dtype='int32')
            # Combiner verticalement les étiquettes associées à chaque phrase afin de les aligner avec les mots d'entrée
            cibles.append(etiquettes)
            # Conserver la position de la phrase en cours, afin de l'utiliser dans l'emplacement des étiquettes de ses mots
            phr_position += 1
            
        elif parties[0]=='A':
            if re.findall("ArtOrDet", parties[2]) or re.findall("Nn", parties[2]) or re.findall("Vt", parties[2]) or re.findall("Prep", parties[2]) or re.findall("Vform", parties[2]) or re.findall("Wform", parties[2]) or re.findall("SVA", parties[2]):
                # Conserver la position du mot erroné qui est extraite de l'annotation associée à la phrase en cours
                digit = [int(j) for j in re.findall("[0-9]+", parties[2][:2])]            
    
                for clé in labels:
                    if  re.search(clé, parties[2]):
                        err = labels.get(clé)
                 
                # En utilisant sa position extraite, placer l'étiquette du mot erroné dans sa phrase
                cibles[phr_position - 1][digit[0]-1] = err

158784it [14:39, 180.50it/s]


In [3]:
phr_lon = []
for phr in phrases:
    phr_lon.append(len(phr.split()))
     
MAX_SEQUENCE_LONGUEUR = max(phr_lon)
print(MAX_SEQUENCE_LONGUEUR) 

222


In [4]:
MAX_VOCAB_TAILLE = len(set(mots))
print(MAX_VOCAB_TAILLE)

33762


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_TAILLE, filters='\t\n')
tokenizer.fit_on_texts(phrases)
sequences = tokenizer.texts_to_sequences(phrases)
word2idx = tokenizer.word_index

In [7]:
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LONGUEUR, padding='post', value=0)

In [8]:
y = pad_sequences(cibles, maxlen=MAX_SEQUENCE_LONGUEUR, padding='post', value=0)

In [9]:
print(X.shape)
print(y.shape)

(57151, 222)
(57151, 222)


In [10]:
# Paramètres de contrôle des expérimentations:
UNITS=  25 #50 #100
EMBEDDING_DIM = 200 #50 #100
BATCH_TAILLE = 16 #32 #64   
DROPOUT_VAL = 0.1 #0 #0.2

# Constantes:
EPOCHS = 3
RECURRENT_DROPOUT_VAL = 0.2

In [11]:
word2vec = {}
with open(os.path.join('/Users/highsierra/Tech/Labortory/ML-Fundamentals/glove.6B/glove.6B.%sd.txt' % EMBEDDING_DIM)) as f:
    for ligne in tqdm(f):
        values = ligne.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

400000it [00:28, 14104.19it/s]


In [12]:
embedding_matrix = np.zeros((MAX_VOCAB_TAILLE,EMBEDDING_DIM))
for word, i in tqdm(word2idx.items()):
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 29347/29347 [00:00<00:00, 100604.03it/s]


## Modélisation

In [13]:
# Répartir les phrases en ensembles de formation et de test selon les pourcentages: 80%, 20%

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=seed)

In [14]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout,Reshape, SimpleRNN, Bidirectional
from keras.utils import to_categorical

In [15]:
def Model_(x_tr, y_tr, epos=3, my_batch_taille=BATCH_TAILLE):  
    input = Input(shape=(MAX_SEQUENCE_LONGUEUR,)) 
    model = Embedding(input_dim=MAX_VOCAB_TAILLE, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LONGUEUR, trainable=False)(input)
    model = LSTM(units=UNITS, return_sequences=True, recurrent_dropout=RECURRENT_DROPOUT_VAL)(model)
    model = LSTM(units=UNITS, return_sequences=True, recurrent_dropout=RECURRENT_DROPOUT_VAL)(model)
    #model = SimpleRNN(units=UNITS, return_sequences=True, recurrent_dropout=RECURRENT_DROPOUT_VAL)(model)
    model = Dropout(DROPOUT_VAL)(model)
    out = TimeDistributed(Dense(8, activation='softmax'))(model)
    model = Model(input, out)
    
    return model

In [16]:
model = Model_(X_train,  y_train)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Pour la formation et le test du modèle dans Keras, nous devons convertir l'ensemble des étiquettes ou des sorties (y) en catégoriels.

ycat_train = to_categorical(y_train, num_classes=8)
ycat_test = to_categorical(y_test, num_classes=8) 

In [19]:
model.fit(X_train, ycat_train, epochs=EPOCHS, batch_size=BATCH_TAILLE, verbose=1) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
45720/45720 [==============================] - 2847s 62ms/step - loss: 0.0307 - accuracy: 0.9980
Epoch 2/3
45720/45720 [==============================] - 2555s 56ms/step - loss: 0.0114 - accuracy: 0.9983
Epoch 3/3
45720/45720 [==============================] - 3135s 69ms/step - loss: 0.0105 - accuracy: 0.9983


## Évaluation
Calculez les mesures suivantes :
1. Accuracy
2. Précision
3. Rappel
4. F1 

In [20]:
print(model.metrics_names)

['loss', 'accuracy']


In [21]:
scores = model.evaluate(X_test, ycat_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

11431/11431 [==============================] - 79s 7ms/step
Accuracy: 99.83%


In [22]:
pred = model.predict(X_test, verbose=1) 
print(pred.shape)

11431/11431 [==============================] - 71s 6ms/step
(11431, 222, 8)


In [23]:
y_pred = np.argmax(pred,axis=-1)

In [24]:
from sklearn.metrics import precision_score, recall_score, f1_score


pres_score = []

for tru,pred in zip (y_test, y_pred):
    pres_score.append(precision_score(tru,pred,average='macro'))

precision = np.mean(pres_score)
print(precision) 

/opt/miniconda3/envs/tensorflow1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8642896614574509


In [25]:
rec_score = []

for tru,pred in zip (y_test, y_pred):
    rec_score.append(recall_score(tru,pred,average='macro'))

rappel = np.mean(rec_score)
print(rappel)

0.8649695273087803


In [26]:
# Manuellement, le F1-score est calculé selon la formule :  f1 = (2 * precision * rappel) / (precision + rappel)
f_score = []

for tru,pred in zip (y_test, y_pred):
    f_score.append(f1_score(tru,pred,average='macro'))
f1 = np.mean(f_score)
print(f1)

0.8646282468000843
